### Build the dataframe from the raw files

In [2]:
# Import required packages
import pandas as pd
import os
import xml.etree.ElementTree as ET
import spacy
import re
from lingua import Language, LanguageDetectorBuilder

In [2]:
def parse_xml(xml_file):
    global empty_counter

    # Load the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Define the namespace
    namespace = {
        'atom': 'http://www.w3.org/2005/Atom',
        'nitf': 'http://iptc.org/std/NITF/2006-10-18/',
        'dc': 'http://purl.org/dc/elements/1.1/'
    }

    # Extract the ID
    nexis_id = root.find('.//atom:id', namespace).text
    nexis_id = nexis_id.split('Item:')[1]
    nexis_id = nexis_id.replace("-", "")

    # Extract the title
    title = root.find('.//atom:title', namespace).text

    # Extract the published date
    published_date = root.find('.//atom:published', namespace).text
    published_date = published_date.split('T')[0]

    # Extract the publisher
    publisher = root.find('.//metadata/publicationInfo/publicationName').text


    # Extract the body text
    body_text = root.find('.//nitf:body.content//bodyText//p', namespace)

    # Extract all body paragraphs and join them into a single line
    body_paragraphs = []
    body_text_element = root.find('.//nitf:body.content//bodyText', namespace)
    if body_text_element is not None:
        for p_element in body_text_element.findall('.//p', namespace):
            if p_element.text:
                body_paragraphs.append(p_element.text.strip())
    text = ' '.join(body_paragraphs)

    # Check for empty text bodies and return
    if text == '':
        #print('No text body in file ' + file_path)
        empty_counter += 1
        return

    # Create a dictionary with the extracted values
    data_dict = {
        'nexis_id': [nexis_id],
        'title': [title],
        'publication_date': [published_date],
        'publisher': [publisher],
        'text': [text]
    }

    df = pd.DataFrame(data_dict)
    return df

In [3]:
# Root Directory of the data
folder_path = 'c:\\Friendcloud\\_University\\_Masterarbeit\\Data\\Nexis\\'

# Initialize empty counter
empty_counter = 0

# Go through all subdirectories and fill raw_data with the data
raw_data = []
for root, _, files in os.walk(folder_path):
    for file in files:
        if file.endswith('.xml'):
            try:
                file_path = os.path.join(root, file)
                data = parse_xml(file_path)
                raw_data.append(data)
            except:
                print('Error in file ' + file_path)

# Build pandas DataFrame
df_raw = pd.concat(raw_data, ignore_index=True)
print('Number of articles removed: ' + str(empty_counter))
print('Number of articles accepted: ' + str(len(df_raw.index)))

Error in file c:\Friendcloud\_University\_Masterarbeit\Data\Nexis\DieZEITinklusiveZEITMagazin\533RXXJ1JDPTM3D40000000_Die_neuen_Weltwunder_Acht_technische.xml


In [ ]:
df_raw

In [ ]:
df_raw.to_pickle("../Data/df_raw.pkl")

### Basic Data preprocessing

In [38]:
df_processed = pd.read_pickle("../Data/df_raw.pkl")

In [39]:
# Add year of publication to dataset
df_processed['publication_date'] = pd.to_datetime(df_processed['publication_date'], errors='coerce')
df_processed['publication_year'] = df_processed['publication_date'].dt.year
df_processed = df_processed[['nexis_id', 'title', 'publication_date', 'publication_year', 'publisher', 'text']]

In [40]:
# Remove articles with duplicate id
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='nexis_id', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 3


In [41]:
# Find examples of articles with duplicate text
df_processed[df_processed.duplicated(subset='text', keep=False)].sample(10)

,nexis_id,title,publication_date,publication_year,publisher,text
876,62YYVY51F13R808M0000000,Eine Frage des Vertrauens Kruschel erklärt´s :...,2021-06-22,2021,Bürstädter Zeitung (Germany),Vor 20 Jahren wurde es eingeführt und dürfte j...
4754,61FCB4T1JCT4R0Y20000000,Produkte mit Hanfextrakt CBD könnten Lebensmit...,2020-12-04,2020,SDA - Basisdienst Deutsch,Hanfprodukte mit Cannabidiol könnten in der Eu...
6681,5X509TF1F13R82190000000,Käfer & Co. als Leckerbissen,2019-09-28,2019,Wiesbadener Tagblatt (Germany),"Hmmm, lecker sieht der Burger aus! So richtig ..."
3985,68MD46T1JBN972CV0000000,"«Wenn eine Kuh geschlachtet wird, schüttelt mi...",2023-07-05,2023,Neue Zürcher Zeitung (Internationale Ausgabe) ...,"Herr Oesch, sind Sie der einzige Metzger in Ih..."
373,6845FTV1JBH6C2JC0000000,Saturday Kitchen - 10:00 AM GMT,2023-04-29,2023,TVEyes - BBC 1 North,This copy is computer generated. Text will var...
3030,5RK9NFW1JDMNJ0570000000,Burger für das Klima und gegen den Hunger,2018-02-06,2018,Lauterbacher Anzeiger (Germany),BERLIN. Das Foto auf der Website des amerikani...
3091,3TF04NM000DBC0WC0000000,None,2023-08-10,2023,Lebensmittel Zeitung,"/vwd. Bruessel, 2. Januar. Der EU-Ministerrat ..."
39,64SXGH11JBN9725X0000000,Bühler hat «eine solide Basis gelegt»,2022-02-16,2022,Appenzeller Zeitung,Dank dicker Auftragsbücher sieht sich der Tech...
6554,61K0CKX1F16N00030000000,Cannabisextrakt,2020-12-21,2020,WELT ONLINE (Deutsch),Der Mitzwanziger suchte schon lange ein natürl...
6628,5C839RS1F13R816T0000000,Chlor- oder Antibiotika-Hähnchen?,2014-05-22,2014,Wiesbadener Kurier (Germany),Wiesbaden . Das geplante Freihandelsabkommen m...


In [42]:
# Remove articles with duplicate text
count_before = len(df_processed.index)
df_processed = df_processed.drop_duplicates(subset='text', keep='first')
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 1278


In [43]:
# Remove articles with length less than 100 characters
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['text'].str.len() > 100]
count_after = len(df_processed.index)
print('Number of articles removed: ' + str(count_before - count_after))

Number of articles removed: 73


In [44]:
# Remove articles which are not in German using lingua
detector = LanguageDetectorBuilder.from_all_spoken_languages().build()

# Add empty language column to dataset
df_processed['language'] = ''

# Detect language of each article
for index, row in df_processed.iterrows():
    text = row['text']
    language = detector.detect_language_of(text)
    df_processed.at[index, 'language'] = language.iso_code_639_1.name

# Print the language and publisher of articles which are not in German
print(df_processed[df_processed['language'] != 'DE']['language'].value_counts())
print(df_processed[df_processed['language'] != 'DE']['publisher'].value_counts())

# Remove articles which are not in German
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['language'] == 'DE']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

language
EN    118
ES      1
Name: count, dtype: int64
publisher
dpa-AFX ProFeed                                                              19
News Bites - People in Business                                              12
American Banking and Market News                                             10
Spiegel Online                                                                7
Industry SnapShot                                                             4
GJAE - German Journal of Agricultural Economics (ehemals Agrarwirtschaft)     4
Agrarwirtschaft                                                               4
Newstex Blogs                                                                 4
Industry SnapShot Summary                                                     3
PR Newswire                                                                   2
MENAFN - Market Reports (English)                                             2
MENAFN - Press Releases (English)                      

In [45]:
# Remove articles with title 'Programmübersicht'
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['title'] != 'Programmübersicht']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

# Remove articles with title 'Programmübersicht Samstag'
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['title'] != 'Programmübersicht Samstag']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

# Remove articles with title 'Programmhinweise'
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['title'] != 'Programmhinweise']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

# Remove articles with specific nexis_ids (tv program)
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['nexis_id'] != '4SC3XK90TXHH10SS0000000']
df_processed = df_processed[df_processed['nexis_id'] != '4SC3XKC0TXHH101J0000000']
df_processed = df_processed[df_processed['nexis_id'] != '4SC3XKJ0TXHH101K0000000']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

# Remove articles with specific nexis_ids (theatre)
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['nexis_id'] != '4JXWDKF0TWRXK1R10000000']
df_processed = df_processed[df_processed['nexis_id'] != '3SGDYHS0006XC4P80000000']
df_processed = df_processed[df_processed['nexis_id'] != '5KNN3CD1JC3P04FD0000000']
df_processed = df_processed[df_processed['nexis_id'] != '4KF5FDD0TWRXK1WY0000000']
df_processed = df_processed[df_processed['nexis_id'] != '585XW971JBPW93GX0000000']
df_processed = df_processed[df_processed['nexis_id'] != '4S4Y46D0TWX2707B0000000']
df_processed = df_processed[df_processed['nexis_id'] != '554MD281F19FX2YB0000000']
df_processed = df_processed[df_processed['nexis_id'] != '4S554KS0TWX271450000000']
df_processed = df_processed[df_processed['nexis_id'] != '518R7XC1F19FX3P10000000']
df_processed = df_processed[df_processed['nexis_id'] != '539ND9G1DYK6Y0950000000']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))

# Remove articles with specific nexis_ids (no article)
count_before = len(df_processed.index)
df_processed = df_processed[df_processed['nexis_id'] != '65S924K1JBR841DW0000000']
count_after = len(df_processed.index)
print('\nNumber of articles removed: ' + str(count_before - count_after))


Number of articles removed: 4

Number of articles removed: 1

Number of articles removed: 2

Number of articles removed: 3

Number of articles removed: 10

Number of articles removed: 1


In [46]:
# Reindex the dataframe
df_processed = df_processed.reset_index(drop=True)

# Remove the language column
df_processed = df_processed.drop(columns=['language'])

In [47]:
df_processed

,nexis_id,title,publication_date,publication_year,publisher,text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...,...,...
5280,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
5281,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
5282,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
5283,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [48]:
df_processed.to_pickle("../Data/df.pkl")

### Text Clean up

In [49]:
VARIANT = "LARGE" # "SMALL", "MEDIUM" , "FULL"

if VARIANT == "SMALL":
    df = pd.read_pickle("../Data/df.pkl")
    df = df.head(20)
elif VARIANT == "MEDIUM":
    df = pd.read_pickle("../Data/df.pkl")
    df = df.head(500)
else:
    df = pd.read_pickle("../Data/df.pkl")

In [50]:
df

,nexis_id,title,publication_date,publication_year,publisher,text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach..."
...,...,...,...,...,...,...
5280,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...
5281,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ..."
5282,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...
5283,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...


In [51]:
# Load the German language model in Spacy
nlp = spacy.load('de_core_news_sm')

# Define a function to clean the text
def clean_text(text):
    # Remove double dashes
    text = text.replace('--', ' ')

    # Remove special characters
    text = re.sub(r"[^A-Za-z0-9äöüÄÖÜß ]+", '', text)
    
    # Lemmatize the text
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_.lower() for token in doc])

    # If the first word has the label "GPE", remove it
    if doc[0].ent_type_ == 'GPE':
        print('GPE found')
        lemmatized_text = lemmatized_text.split(' ', 1)[1]
        
    return lemmatized_text

# Apply the clean_text function to the 'text' column in df_processed and save as a new dataframe df_clean
df_full = df.copy()
df_full['clean_text'] = df_full['text'].apply(clean_text)

In [52]:
df_full

,nexis_id,title,publication_date,publication_year,publisher,text,clean_text
0,5MNMH621JB0GF09H0000000,Angst vor dem «harten Brexit» auf der Insel - ...,2017-01-15,2017,AP Deutsch,Wenn Großbritannien Ende März den Ausstieg aus...,wenn großbritannien ende märz der ausstieg aus...
1,5SM3THB1DXFJ50MP0000000,Fleischindustrie wehrt sich gegen Marketing fü...,2018-06-21,2018,AP Deutsch,Fleisch aus dem Labor ist noch eine Zukunftsvi...,fleisch aus der labor sein noch ein zukunftsvi...
2,5SM3THB1DXFJ50MY0000000,Der AP-Überblick am Nachmittag,2018-06-21,2018,AP Deutsch,Die AP Weltnachrichten haben heute unter ander...,der ap weltnachrichten haben heute unter ander...
3,5STNVWH1DXFJ53VM0000000,Laborfleisch soll in drei Jahren auf die Telle...,2018-07-17,2018,AP Deutsch,Maastricht (AP) - Das niederländische Unterneh...,maastricht ap der niederländisch unternehmen...
4,5VHK2XG1JB0GF4Y50000000,Israelische Forscher wollen künstliche Steaks ...,2019-02-26,2019,AP Deutsch,"Die Weltbevölkerung wächst, die Nachfrage nach...",der weltbevölkerung wachsen der nachfrage nach...
...,...,...,...,...,...,...,...
5280,67KW1VK1F15WB4660000000,Kein Titel,2023-02-21,2023,ZEIT Wissen,6 Am anfang drei Fragen 1. Können wir andere m...,6 an anfang drei frage 1 können wir anderer mi...
5281,67KW1VK1F15WB46B0000000,Leben und schmecken lassen,2023-02-21,2023,ZEIT Wissen,"Ein saftiges Filet, für das kein Huhn sterben ...",ein saftig filet für der kein huhn sterben mus...
5282,7W29GN20YC2460S30000000,ABSCHIED VOM ALTEN ITALIEN,2009-05-13,2009,ZEIT Wissen,John Dickie: »Delizia! Die Italiener und ihre...,john dickie delizia der italiener und ihr kü...
5283,7X8DW4712SK2G0H10000000,Essen aus dem Labor,2009-12-08,2009,ZEIT Wissen,Es ist der letzte Tag auf der Lebensmittelmess...,es sein der letzter tag auf der lebensmittelme...


In [53]:
df_full.to_pickle("../Data/df_full.pkl")